In [ ]:
import os
import pandas as pd
import numpy as np
import config, models
from itertools import permutations

In [ ]:
# main variables
dataset_name               = "FB13"
embedding_model            = models.TransD
model_timestamp            = '1524257330'

# classification settings
label_original_graph = True
predict_whole_graph = False

# device settings: GPU and CPU
work_threads               = 4
cuda_visible_devices       = "0" # should be a string

## Restore working model

In [ ]:
import_path = './results/{}/{}/{}/'.format(
    dataset_name,
    embedding_model.__name__,
    model_timestamp
)

model_info_df = pd.read_csv('{}/model_info.tsv'.format(import_path), sep='\t')
model_info_df.transpose()

In [ ]:
# transform model info into dict with only one "row"
model_info = model_info_df.to_dict()
for key,d in model_info.iteritems():
    model_info[key] = d[0]

In [ ]:
con = config.Config()
dataset_path = "./benchmarks/{}/".format(model_info['dataset_name'])
con.set_in_path(dataset_path)
con.set_test_link_prediction(False)
con.set_test_triple_classification(True)
con.set_work_threads(work_threads)
con.set_dimension(int(model_info['k']))
con.init()
con.set_model(embedding_model,
              score_norm=model_info['score_norm'])
con.import_variables("{}tf_model/model.vec.tf".format(import_path)) # loading model via tensor library

## Read dataset

In [ ]:
train = pd.read_csv(dataset_path + 'train2id.txt', sep=' ', skiprows=1, names=['e1', 'e2', 'rel'])
valid = pd.read_csv(dataset_path + 'valid2id.txt', sep=' ', skiprows=1, names=['e1', 'e2', 'rel'])
test = pd.read_csv(dataset_path + 'test2id.txt', sep=' ', skiprows=1, names=['e1', 'e2', 'rel'])

valid_neg = pd.read_csv(dataset_path + 'valid2id_neg.txt', sep=' ', skiprows=1, names=['e1', 'e2', 'rel'])
test_neg = pd.read_csv(dataset_path + 'test2id_neg.txt', sep=' ', skiprows=1, names=['e1', 'e2', 'rel'])

data = pd.concat([train, valid, test])

# training with corrupted triples
train_corrupted = pd.read_csv(dataset_path + 'corrupted/train2id.txt')

## Classify

In [ ]:
con.classify([1], [3], [0])

## Debugging